# Dependencies

In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
import joblib
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

In [4]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

# Preprocess the raw data

### Read the CSV

In [5]:
df = pd.read_csv("Resources/exoplanet_data.csv")

### Perform Basic Data Cleaning

In [6]:
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [7]:
df.columns

Index(['koi_disposition', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 'koi_period_err1', 'koi_period_err2',
       'koi_time0bk', 'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact',
       'koi_impact_err1', 'koi_impact_err2', 'koi_duration',
       'koi_duration_err1', 'koi_duration_err2', 'koi_depth', 'koi_depth_err1',
       'koi_depth_err2', 'koi_prad', 'koi_prad_err1', 'koi_prad_err2',
       'koi_teq', 'koi_insol', 'koi_insol_err1', 'koi_insol_err2',
       'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_steff_err1',
       'koi_steff_err2', 'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2',
       'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 'ra', 'dec',
       'koi_kepmag'],
      dtype='object')

### Select significant features (columns)
Perform feature selection based upon physical characteristics of the exoplanet candidates. These will also be used as X values.<br>I will mainly select features related to the appearance and physical characteristics. They would be checked off in the following lists. Also, if there is a feature that might result from a linear combination of other features, then that would be unchecked off.

KOI = Kepler Objects of Interest<br> It is a number used to identify and track a Kepler Object of Interest. A KOI is a target identified by the Kepler Project that displays at least one transit-like sequence within Kepler time-series photometry that appears to be of astrophysical origin and initially consistent with a planetary transit hypothesis.

#### Project Disposition Columns
Flags designate the most probable physical explanation of the KOI.

- [x] koi_fpflag_nt: Not Transit-Like Flag - KOI whose light curve is not consistent with that of a transiting planet.
- [x] koi_fpflag_ss: Stellar Eclipse Flag - A KOI that is observed to have a significant secondary event, transit shape, or out-of-eclipse variability.
- [x] koi_fpflag_co: Centroid Offset Flag - The source of the signal is from a nearby star.
- [x] koi_fpflag_ec: Ephemeris Match Indicates Contamination Flag - The KOI shares the same period and epoch as another object.

#### Transit Properties
Transit parameters delivered by the Kepler Project are typically best-fit parameters produced by a Mandel-Agol (2002) fit to a multi-quarter Kepler light curve, assuming a linear orbital ephemeris. Some of the parameters listed below are fit directly, other are derived from the best-fit parameters.

- [x] koi_period: Orbital Period (days) - The interval between consecutive planetary transits.
- [x] koi_period_err1: Orbital Period (days) - Uncertainties Column (positive +)
- [ ] koi_period_err2: Orbital Period (days) - Uncertainties Column (negative -)
- [x] koi_time0bk: Transit Epoch - The time corresponding to the center of the first detected transit in Barycentric Julian Day (BJD) minus a constant offset of 2,454,833.0 days.
- [x] koi_time0bk_err1: Transit Epoch - Uncertainties Column (positive +)
- [ ] koi_time0bk_err2: Transit Epoch - Uncertainties Column (negative -)
- [x] koi_impact: Impact Parameter - The sky-projected distance between the center of the stellar disc and the center of the planet disc at conjunction, normalized by the stellar radius.
- [x] koi_impact_err1: Impact Parameter - Uncertainties Column (positive +)
- [ ] koi_impact_err2: Impact Parameter - Uncertainties Column (negative -)
- [x] koi_duration: Transit Duration (hours) - The duration of the observed transits.
- [x] koi_duration_err1: Transit Duration (hours) - Uncertainties Column (positive +)
- [ ] koi_duration_err2: Transit Duration (hours) - Uncertainties Column (negative -)
- [x] koi_depth: Transit Depth (parts per million) - The fraction of stellar flux lost at the minimum of the planetary transit.
- [x] koi_depth_err1: Transit Depth (parts per million) - Uncertainties Column (positive +)
- [ ] koi_depth_err2: Transit Depth (parts per million) - Uncertainties Column (negative -)
- [x] koi_prad: Planetary Radius (Earth radii) - The radius of the planet. Planetary radius is the product of the planet star radius ratio and the stellar radius.
- [x] koi_prad_err1: Planetary Radius (Earth radii) - Uncertainties Column (positive +)
- [x] koi_prad_err2: Planetary Radius (Earth radii) - Uncertainties Column (negative -)
- [x] koi_teq: Equilibrium Temperature (Kelvin) - Approximation for the temperature of the planet.
- [x] koi_insol: Insolation Flux [Earth flux]
- [x] koi_insol_err1: Insolation Flux [Earth flux] - Uncertainties Column (positive +)
- [x] koi_insol_err2: Insolation Flux [Earth flux] - Uncertainties Column (negative -)

#### Threshold-Crossing Event (TCE) Information
The Transiting Planet Search (TPS) module of the Kepler data analysis pipeline performs a detection test for planet transits in the multi-quarter, gap-filled flux time series. The TPS module detrends each quarterly PDC light curve to remove edge effects around data gaps and then combines the data segments together, filling gaps with interpolated data so as to condition the flux time series for a matched filter. 

- [x] koi_model_snr: Transit Signal-to-Noise - Transit depth normalized by the mean uncertainty in the flux during the transits.
- [x] koi_tce_plnt_num: TCE Planet Number - TCE Planet Number federated to the KOI.

#### Stellar Parameters
Stellar effective temperature, surface gravity, metallicity, radius, mass, and age should comprise a consistent set.

- [x] koi_steff: Stellar Effective Temperature (Kelvin) - The photospheric temperature of the star.
- [x] koi_steff_err1: Stellar Effective Temperature (Kelvin) - Uncertainties Column (positive +)
- [x] koi_steff_err2: Stellar Effective Temperature (Kelvin) - Uncertainties Column (negative -)
- [x] koi_slogg: Stellar Surface Gravity - The base-10 logarithm of the acceleration due to gravity at the surface of the star.
- [x] koi_slogg_err1: Stellar Surface Gravity - Uncertainties Column (positive +)
- [x] koi_slogg_err2: Stellar Surface Gravity - Uncertainties Column (negative -)
- [x] koi_srad: Stellar Radius (solar radii) - The photospheric radius of the star.
- [x] koi_srad_err1: Stellar Radius (solar radii) - Uncertainties Column (positive +)
- [x] koi_srad_err2: Stellar Radius (solar radii) - Uncertainties Column (negative -)

#### Kepler Input Catalog (KIC) Parameters

- [x] ra: RA (deg) - KIC Right Ascension of the planetary system in decimal degrees
- [x] dec: Dec (deg) - 	KIC Declination in decimal degrees
- [x] koi_kepmag: Kepler-band (mag) - Kepler-band (mag), it is a magnitude computed according to a hierarchical scheme and depends on what pre-existing catalog source is available.

In [8]:
# Selected features
feature_names = ['koi_fpflag_nt', 
        'koi_fpflag_ss',
        'koi_fpflag_co',
        'koi_fpflag_ec',
        'koi_period',
        'koi_period_err1',
        'koi_time0bk',
        'koi_time0bk_err1',
        'koi_impact',
        'koi_impact_err1',
        'koi_duration',
        'koi_duration_err1',
        'koi_depth',
        'koi_depth_err1',
        'koi_prad',
        'koi_prad_err1',
        'koi_prad_err2',
        'koi_teq',
        'koi_insol',
        'koi_insol_err1',
        'koi_insol_err2',
        'koi_model_snr',
        'koi_tce_plnt_num',
        'koi_steff',
        'koi_steff_err1',
        'koi_steff_err2',
        'koi_slogg',
        'koi_slogg_err1',
        'koi_slogg_err2',
        'koi_srad',
        'koi_srad_err1',
        'koi_srad_err2',
        'ra',
        'dec',
        'koi_kepmag'
       ]
X = df[feature_names]
X.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_time0bk,koi_time0bk_err1,koi_impact,koi_impact_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,0,0,0,0,54.418383,2.479000e-04,162.513840,0.003520,0.586,0.059,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,0,1,0,0,19.899140,1.490000e-05,175.850252,0.000581,0.969,5.126,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,0,1,0,0,1.736952,2.630000e-07,170.307565,0.000115,1.276,0.115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,0,0,0,0,2.525592,3.760000e-06,171.595550,0.001130,0.701,0.235,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,0,0,0,0,4.134435,1.050000e-05,172.979370,0.001900,0.762,0.139,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


Use `koi_disposition` for the y values

In [9]:
y = df["koi_disposition"]
y.head()

0         CONFIRMED
1    FALSE POSITIVE
2    FALSE POSITIVE
3         CONFIRMED
4         CONFIRMED
Name: koi_disposition, dtype: object

### Label-encode y column

In [10]:
label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)

In [11]:
for label, original_class in zip(encoded_y, y):
    print('Original Class: ' + str(original_class))
    print('Encoded Label: ' + str(label))
    print('-' * 12)

Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: CO

Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: CANDIDATE
Encoded Label: 0
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CANDIDATE
Encoded Label: 0
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Ori

Encoded Label: 1
------------
Original Class: CANDIDATE
Encoded Label: 0
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CANDIDATE
Encoded Label: 0
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CANDIDATE
Encoded Label: 0
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CANDIDATE
Encoded Label: 0
------------
Original

Encoded Label: 2
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CANDIDATE
Encoded Label: 0
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CO

------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CANDIDATE
Encoded Label: 0
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CANDIDATE
Encoded Label: 0
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CANDIDATE
Encoded Label: 0
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: CANDIDATE
Encoded Label: 0
------------
Original Class: CANDIDATE
Encoded Label: 0
------------
Original Class: CANDIDATE
Encoded Label: 0
------------
Original Class: CANDIDATE
Encoded Label: 0
------------
Original Class: CANDIDATE
Encoded Label: 0
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CONF

...

------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: CANDIDATE
Encoded Label: 0
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: CANDIDATE
Encoded Label: 0
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: CANDIDATE
Encoded Label: 0
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: CANDIDATE
Encoded Label: 0
------------
Original Class: CANDIDATE
Encod

------------
Original Class: CANDIDATE
Encoded Label: 0
------------
Original Class: CANDIDATE
Encoded Label: 0
------------
Original Class: CANDIDATE
Encoded Label: 0
------------
Original Class: CANDIDATE
Encoded Label: 0
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE


Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: CANDIDATE
Encoded Label: 0
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: CANDIDATE
Encoded Label: 0
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: CANDIDATE
Encoded Label: 0
-

Encoded Label: 0
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: CANDIDATE
Encoded Label: 0
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: CANDIDATE
Encoded Label: 0
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: CANDIDATE
Encoded Label: 0
------------
Original Class: CANDIDATE
Encoded Label: 0
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class

Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: CANDIDATE
Encoded Label: 0
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: CANDIDATE
Encoded Label: 0
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: CANDIDATE
Encoded Label: 0
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original 

In [12]:
# decoded_y = label_encoder.inverse_transform(encoded_y)
decoded_y = label_encoder.inverse_transform([0,1,2])
decoded_y

array(['CANDIDATE', 'CONFIRMED', 'FALSE POSITIVE'], dtype=object)

Note that each of the original labels has been replaced with an integer.

### Spliting the data into training and testing data.

In [13]:
# split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, encoded_y, random_state=42)
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_time0bk,koi_time0bk_err1,koi_impact,koi_impact_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
6122,0,0,0,0,6.768901,7.380000e-05,133.077240,0.008440,0.150,0.305,...,-171,4.327,0.153,-0.187,1.125,0.310,-0.207,294.40472,39.351681,14.725
6370,0,1,0,1,0.733726,6.060000e-06,132.020050,0.007950,0.291,0.193,...,-175,4.578,0.033,-0.187,0.797,0.211,-0.056,284.50391,42.463860,15.770
2879,1,0,0,0,7.652707,6.540000e-05,134.460380,0.006190,0.970,0.879,...,-189,4.481,0.050,-0.200,0.963,0.290,-0.097,295.50211,38.983540,13.099
107,0,0,0,0,7.953547,1.910000e-05,174.662240,0.001820,0.300,0.145,...,-85,4.536,0.056,-0.016,0.779,0.023,-0.049,291.15878,40.750271,15.660
29,0,0,0,0,4.959319,5.150000e-07,172.258529,0.000083,0.831,0.016,...,-77,4.359,0.110,-0.110,1.082,0.173,-0.130,292.16705,48.727589,15.263


### Applying One-Hot Encoding

In [14]:
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)
y_train_categorical

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

### Scaling the data

Scaling the data using the MinMaxScaler

In [15]:
# Scale your data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [16]:
X.shape

(6991, 35)

In [17]:
y.shape

(6991,)

# Original Model

### Create the model: for categorical data, we use a classifier model

In [18]:
number_inputs = 35
number_hidden_nodes_level1 = 40
number_hidden_nodes_level2 = 50
number_classes = 3 # output nodes
# Function to create model, required for KerasClassifier
def create_DeepLearning_model():
    # We first need to create a sequential model
    model = Sequential()
    # add layers
    # Next, we add our first layer. This layer requires us to specify both the number of inputs and 
    # the number of nodes that we want in the hidden layer.
    model.add(Dense(units=number_hidden_nodes_level1,activation='relu', input_dim=number_inputs))
    model.add(Dense(units=number_hidden_nodes_level2,activation='relu'))
    # Our final layer is the output layer. Here, we need to specify the activation function 
    # (typically softmax for classification) and the number of classes (labels) that we are trying to predict
    model.add(Dense(units=number_classes, activation='softmax'))   
    # Compile model: categorical crossentropy for categorical data 
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
# model = create_DeepLearning_model(number_inputs, number_hidden_nodes_level1, number_hidden_nodes_level2, number_classes)
model = create_DeepLearning_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 40)                1440      
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2050      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 153       
Total params: 3,643
Trainable params: 3,643
Non-trainable params: 0
_________________________________________________________________


### Train the Model
Training consists of updating our weights using our optimizer and loss function. In this case, we choose 50 iterations (loops) of training that are called epochs. We also choose to shuffle our training data and increase the detail printed out during each training cycle.

In [20]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=50,
    shuffle=True,
    verbose=2
)

Epoch 1/50
164/164 - 6s - loss: 0.6752 - accuracy: 0.6716
Epoch 2/50
164/164 - 1s - loss: 0.3822 - accuracy: 0.8028
Epoch 3/50
164/164 - 1s - loss: 0.3541 - accuracy: 0.8220
Epoch 4/50
164/164 - 1s - loss: 0.3435 - accuracy: 0.8285
Epoch 5/50
164/164 - 1s - loss: 0.3339 - accuracy: 0.8354
Epoch 6/50
164/164 - 1s - loss: 0.3276 - accuracy: 0.8383
Epoch 7/50
164/164 - 1s - loss: 0.3261 - accuracy: 0.8426
Epoch 8/50
164/164 - 1s - loss: 0.3179 - accuracy: 0.8459
Epoch 9/50
164/164 - 1s - loss: 0.3154 - accuracy: 0.8531
Epoch 10/50
164/164 - 1s - loss: 0.3091 - accuracy: 0.8556
Epoch 11/50
164/164 - 1s - loss: 0.3050 - accuracy: 0.8577
Epoch 12/50
164/164 - 1s - loss: 0.3045 - accuracy: 0.8571
Epoch 13/50
164/164 - 1s - loss: 0.3000 - accuracy: 0.8613
Epoch 14/50
164/164 - 1s - loss: 0.2998 - accuracy: 0.8589
Epoch 15/50
164/164 - 1s - loss: 0.2954 - accuracy: 0.8667
Epoch 16/50
164/164 - 1s - loss: 0.2904 - accuracy: 0.8676
Epoch 17/50
164/164 - 1s - loss: 0.2899 - accuracy: 0.8684
Epoch 

We use our testing data to validate our model. This is how we determine the validity of our model (i.e. the ability to predict new and previously unseen data points)

In [21]:
# Evaluate the original model using the testing data
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Loss: {round(model_loss*100,3)}%, Accuracy: {round(model_accuracy*100,3)}%")

55/55 - 3s - loss: 0.2765 - accuracy: 0.8942
Loss: 27.651%, Accuracy: 89.416%


# Hyperparameter Tuning

### Create the `GridSearchCV` model to find best/tuned parameters

In [22]:
model_keras = KerasClassifier(build_fn=create_DeepLearning_model, verbose=0)
# define the grid search parameters
batch_size = [5, 10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100, 1000]
param_grid = dict(batch_size=batch_size, epochs=epochs)

grid_model = GridSearchCV(model_keras, param_grid, verbose=1, cv=3, n_jobs=-1)

# Train the model with GridSearch
_ = grid_model.fit(X_train_scaled, y_train_categorical)

Fitting 3 folds for each of 28 candidates, totalling 84 fits


### Find Tuned parameters

In [23]:
print(f"Best Parameters: {grid_model.best_params_}")
print(f"Best Score: {round(grid_model.best_score_*100,3)}%")

Best Parameters: {'batch_size': 10, 'epochs': 100}
Best Score: 88.804%


# Tuned Model

### Create the model

In [26]:
# Tuned model based upon best parameters previously found
tuned_model = model

### Train the Model with specific parameters

In [28]:
# Fit and score the tuned model
tuned_model.fit(
    X_train_scaled,
    y_train_categorical,
    batch_size=grid_model.best_params_['batch_size'],
    epochs=grid_model.best_params_['epochs'],
    shuffle=True,
    verbose=2
)

Epoch 1/100
525/525 - 0s - loss: 0.1629 - accuracy: 0.9287
Epoch 2/100
525/525 - 0s - loss: 0.1628 - accuracy: 0.9252
Epoch 3/100
525/525 - 0s - loss: 0.1594 - accuracy: 0.9258
Epoch 4/100
525/525 - 0s - loss: 0.1598 - accuracy: 0.9270
Epoch 5/100
525/525 - 0s - loss: 0.1615 - accuracy: 0.9270
Epoch 6/100
525/525 - 0s - loss: 0.1590 - accuracy: 0.9294
Epoch 7/100
525/525 - 0s - loss: 0.1581 - accuracy: 0.9289
Epoch 8/100
525/525 - 0s - loss: 0.1574 - accuracy: 0.9306
Epoch 9/100
525/525 - 0s - loss: 0.1608 - accuracy: 0.9277
Epoch 10/100
525/525 - 0s - loss: 0.1609 - accuracy: 0.9277
Epoch 11/100
525/525 - 0s - loss: 0.1570 - accuracy: 0.9287
Epoch 12/100
525/525 - 0s - loss: 0.1569 - accuracy: 0.9306
Epoch 13/100
525/525 - 0s - loss: 0.1643 - accuracy: 0.9298
Epoch 14/100
525/525 - 0s - loss: 0.1576 - accuracy: 0.9281
Epoch 15/100
525/525 - 0s - loss: 0.1550 - accuracy: 0.9300
Epoch 16/100
525/525 - 0s - loss: 0.1558 - accuracy: 0.9315
Epoch 17/100
525/525 - 0s - loss: 0.1538 - accura

In [32]:
# Evaluate the tuned model using the testing data
model_loss, model_accuracy = tuned_model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Loss: {round(model_loss*100,3)}%, Accuracy: {round(model_accuracy*100,3)}%")

55/55 - 0s - loss: 0.4934 - accuracy: 0.8902
Loss: 49.341%, Accuracy: 89.016%


### Classification report

In [33]:
# Make predictions with the hypertuned model
predictions = tuned_model.predict(X_test_scaled)

# Transform catgorical arrays to numeric representations
def uncategorize(list):
    pos=0
    for value in list:
        if round(value,0)==1:
            return pos
        pos +=1
    return 0
predictions_uncategorized = [uncategorize(list) for list in predictions]

# Calculate classification report
print(classification_report(y_test, predictions_uncategorized,target_names=["CANDIDATE","FALSE POSITIVE","CONFIRMED"]))

                precision    recall  f1-score   support

     CANDIDATE       0.77      0.81      0.79       411
FALSE POSITIVE       0.83      0.79      0.81       484
     CONFIRMED       0.98      0.99      0.99       853

      accuracy                           0.89      1748
     macro avg       0.86      0.86      0.86      1748
  weighted avg       0.89      0.89      0.89      1748

